In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from fairness.pre_processing import categorical_to_numeric_converter
from sklearn.model_selection import GridSearchCV
import numpy as np

In [2]:
dataset = pd.read_csv('./dataset/ull/ULL_dataset.csv', sep=',')

In [3]:
print("Columns of the dataset before fairness through unawareness: ", len(dataset.columns))

Columns of the dataset before fairness through unawareness:  549


In [4]:
dataset = categorical_to_numeric_converter(dataset)

In [5]:
output_columns = ['level_mat', 'level_len', 'level_ing']

In [6]:
protected_attributes = ['score_MAT', 'level_MAT', 'score_LEN', 'level_LEN', 'score_ING',
       'level_ING', 'a1', 'a4', 'repeater', 'a24', 'a41', 'a42',
       'country_iso_cnac', 'country_iso_nac', 'island', 'capital_island',
       'public_private', 'f3a', 'f3b', 'mother_education',
       'father_education', 'f4a', 'f4b', 'f5a', 'f5b', 'f5n', 'inmigrant',
       'inmigrant2', 'inmigrant_second_gen', 'f11', 'books', 'f23',
       'f24a', 'f24b', 'mother_occupation', 'father_occupation', 'f34',
       'household_income_q', 'escs']

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

random_forest_classifier = RandomForestClassifier()
decision_tree_classifier = DecisionTreeClassifier()
knn_classifier = KNeighborsClassifier()

models = [
    random_forest_classifier,
    decision_tree_classifier,
    knn_classifier
]

random_forest_params = {
    'min_samples_leaf': [5, 7, 9, 11],
    'n_estimators': [200, 500],
    'max_depth' : [10, 20, 50, 80, 100, 150],
    'criterion' :['gini', 'entropy']
}

decision_tree_params = {
    'min_samples_leaf': [5, 7, 9, 11],
    'max_depth' : [10, 20, 50, 80, 100, 150],
    'criterion' :['gini', 'entropy']
}

knn_params = {
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': range(20, 50),
}


model_params = [random_forest_params, decision_tree_params, knn_params]

In [8]:
for output_column in output_columns:
    final_protected_attributes =  []
    if output_column in protected_attributes:
        final_protected_attributes = protected_attributes.remove(output_column)
    else:
        final_protected_attributes = protected_attributes
    dataset.drop(columns=final_protected_attributes, inplace=True)
    print("Columns of the dataset after fairness through unawareness: ", len(dataset.columns))
    
    X = dataset.loc[:, dataset.columns!=output_column]
    y = dataset[output_column]
    y = np.ravel(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    standard_scaler_train = StandardScaler()
    X_train = standard_scaler_train.fit_transform(X_train)

    standard_scaler_test = StandardScaler()
    X_test = standard_scaler_test.fit_transform(X_test)

    best_params = None
    max_accuracy = 0
    best_model = None

    for model_index in range(0, len(models)):
        
        grid_search = GridSearchCV(
        estimator=models[model_index],
        param_grid=model_params[model_index],
        scoring='accuracy', 
        return_train_score=True,
        cv=3
        )

        grid_search_classifier = grid_search.fit(X_train, y_train)
        y_pred = grid_search_classifier.predict(X_test)
        if accuracy_score(y_test, y_pred) > max_accuracy:
            max_accuracy = accuracy_score(y_test, y_pred)
            best_model = models[model_index]
            best_params = model_params[model_index]
    
    print("OUTPUT COLUMN: ", output_column)
    print("Best model: ", best_model)
    print("Best_params: ", best_params)
    print("Accuracy: ", max_accuracy)

ValueError: list.remove(x): x not in list